# Capitulo 4: Separar Datos de Instrucciones

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu API key y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacenamiento de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## Leccion

A menudo, no queremos escribir prompts completos, sino que queremos **plantillas de prompt que puedan ser modificadas posteriormente con datos de entrada adicionales antes de enviarlas a Claude**. Esto puede ser util si quieres que Claude haga lo mismo cada vez, pero los datos que Claude usa para su tarea pueden ser diferentes cada vez.

Afortunadamente, podemos hacer esto bastante facilmente **separando el esqueleto fijo del prompt de la entrada variable del usuario, y luego sustituyendo la entrada del usuario en el prompt** antes de enviar el prompt completo a Claude.

A continuacion, recorreremos paso a paso como escribir una plantilla de prompt sustituible, asi como sustituir la entrada del usuario.

### Ejemplos

En este primer ejemplo, le pedimos a Claude que actue como un generador de sonidos de animales. Observa que el prompt completo enviado a Claude es simplemente el `PROMPT_TEMPLATE` sustituido con la entrada (en este caso, "Cow"). Observa que la palabra "Cow" reemplaza el marcador de posicion `ANIMAL` mediante un f-string cuando imprimimos el prompt completo.

**Nota:** No tienes que llamar a tu variable de marcador de posicion de ninguna manera en particular en la practica. La llamamos `ANIMAL` en este ejemplo, pero con la misma facilidad, podriamos haberla llamado `CREATURE` o `A` (aunque generalmente es bueno que los nombres de tus variables sean especificos y relevantes para que tu plantilla de prompt sea facil de entender incluso sin la sustitucion, solo por legibilidad). Solo asegurate de que el nombre que le des a tu variable sea el que uses para el f-string de la plantilla de prompt.

In [ ]:
# Contenido variable
ANIMAL = "Cow"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

Por que querriamos separar y sustituir entradas de esta manera? Bueno, **las plantillas de prompt simplifican las tareas repetitivas**. Digamos que construyes una estructura de prompt que invita a usuarios externos a enviar contenido al prompt (en este caso, el animal cuyo sonido quieren generar). Estos usuarios externos no tienen que escribir ni siquiera ver el prompt completo. Todo lo que tienen que hacer es completar las variables.

Hacemos esta sustitucion aqui usando variables y f-strings, pero tambien puedes hacerlo con el metodo format().

**Nota:** Las plantillas de prompt pueden tener tantas variables como se desee!

Al introducir variables de sustitucion como estas, es muy importante **asegurarse de que Claude sepa donde comienzan y terminan las variables** (vs. las instrucciones o descripciones de tareas). Veamos un ejemplo donde no hay separacion entre las instrucciones y la variable de sustitucion.

Para nuestros ojos humanos, es muy claro donde comienza y termina la variable en la plantilla de prompt a continuacion. Sin embargo, en el prompt completamente sustituido, esa delimitacion se vuelve poco clara.

In [ ]:
# Contenido variable
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

Aqui, **Claude piensa que "Yo Claude" es parte del correo electronico que se supone debe reescribir**! Puedes notarlo porque comienza su reescritura con "Dear Claude". Para el ojo humano, es claro, particularmente en la plantilla de prompt donde comienza y termina el correo electronico, pero se vuelve mucho menos claro en el prompt despues de la sustitucion.

Como resolvemos esto? **Envuelve la entrada en etiquetas XML**! Lo hicimos a continuacion, y como puedes ver, ya no hay "Dear Claude" en la salida.

Las [etiquetas XML](https://docs.anthropic.com/claude/docs/use-xml-tags) son etiquetas con corchetes angulares como `<tag></tag>`. Vienen en pares y consisten en una etiqueta de apertura, como `<tag>`, y una etiqueta de cierre marcada con un `/`, como `</tag>`. Las etiquetas XML se usan para envolver contenido, asi: `<tag>contenido</tag>`.

**Nota:** Aunque Claude puede reconocer y trabajar con una amplia gama de separadores y delimitadores, recomendamos que **uses especificamente etiquetas XML como separadores** para Claude, ya que Claude fue entrenado especificamente para reconocer las etiquetas XML como un mecanismo de organizacion de prompts. Fuera de las llamadas a funciones, **no hay etiquetas XML especiales con las que Claude haya sido entrenado que debas usar para maximizar tu rendimiento**. Hemos hecho a Claude muy maleable y personalizable de esta manera a proposito.

In [ ]:
# Contenido variable
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

Veamos otro ejemplo de como las etiquetas XML pueden ayudarnos.

En el siguiente prompt, **Claude interpreta incorrectamente que parte del prompt es la instruccion vs. la entrada**. Incorrectamente considera que `Each is about an animal, like rabbits` es parte de la lista debido al formato, cuando el usuario (el que completa la variable `SENTENCES`) presumiblemente no queria eso.

In [ ]:
# Contenido variable
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

Para solucionar esto, solo necesitamos **rodear las oraciones de entrada del usuario con etiquetas XML**. Esto le muestra a Claude donde comienzan y terminan los datos de entrada a pesar del guion engañoso antes de `Each is about an animal, like rabbits.`

In [ ]:
# Contenido variable
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

**Nota:** En la version incorrecta del prompt "Each is about an animal", tuvimos que incluir el guion para que Claude respondiera incorrectamente de la manera que queriamos para este ejemplo. Esta es una leccion importante sobre prompting: **los pequenos detalles importan**! Siempre vale la pena **revisar tus prompts en busca de errores tipograficos y gramaticales**. Claude es sensible a los patrones (en sus primeros años, antes del ajuste fino, era una herramienta de prediccion de texto sin procesar), y es mas probable que cometa errores cuando tu cometes errores, mas inteligente cuando suenas inteligente, mas tonto cuando suenas tonto, y asi sucesivamente.

Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 4.1 - Tema del Haiku](#ejercicio-41---tema-del-haiku)
- [Ejercicio 4.2 - Pregunta sobre Perros con Errores](#ejercicio-42---pregunta-sobre-perros-con-errores)
- [Ejercicio 4.3 - Pregunta sobre Perros Parte 2](#ejercicio-43---pregunta-sobre-perros-parte-2)

### Ejercicio 4.1 - Tema del Haiku
Modifica el `PROMPT` para que sea una plantilla que tome una variable llamada `TOPIC` y genere un haiku sobre ese tema. Este ejercicio solo pretende evaluar tu comprension de la estructura de plantillas con variables usando f-strings.

In [ ]:
# Contenido variable
TOPIC = "Pigs"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f""

# Obtener la respuesta de Claude
response = get_completion(PROMPT)

# Funcion para calificar si el ejercicio es correcto
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(response)
print("\n------------------------------------------ CALIFICACION ------------------------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### Ejercicio 4.2 - Pregunta sobre Perros con Errores
Corrige el `PROMPT` añadiendo etiquetas XML para que Claude produzca la respuesta correcta.

Intenta no cambiar nada mas del prompt. La escritura desordenada y llena de errores es intencional, para que puedas ver como reacciona Claude ante tales errores.

In [ ]:
# Contenido variable
QUESTION = "ar cn brown?"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Obtener la respuesta de Claude
response = get_completion(PROMPT)

# Funcion para calificar si el ejercicio es correcto
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(response)
print("\n------------------------------------------ CALIFICACION ------------------------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### Ejercicio 4.3 - Pregunta sobre Perros Parte 2
Corrige el `PROMPT` **SIN** añadir etiquetas XML. En su lugar, elimina solo una o dos palabras del prompt.

Al igual que con los ejercicios anteriores, intenta no cambiar nada mas del prompt. Esto te mostrara que tipo de lenguaje Claude puede analizar y entender.

In [ ]:
# Contenido variable
QUESTION = "ar cn brown?"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Obtener la respuesta de Claude
response = get_completion(PROMPT)

# Funcion para calificar si el ejercicio es correcto
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(response)
print("\n------------------------------------------ CALIFICACION ------------------------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

### Felicidades!

Si has resuelto todos los ejercicios hasta este punto, estas listo para pasar al siguiente capitulo. Feliz prompting!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompts mostrados en esta leccion y modifiques los prompts para ver como pueden afectar las respuestas de Claude.

In [ ]:
# Contenido variable
ANIMAL = "Cow"

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Contenido variable
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Contenido variable
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Contenido variable
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Contenido variable
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Plantilla de prompt con un marcador de posicion para el contenido variable
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Imprimir la respuesta de Claude
print("--------------------------- Prompt completo con sustituciones de variables ---------------------------")
print(PROMPT)
print("\n------------------------------------- Respuesta de Claude -------------------------------------")
print(get_completion(PROMPT))